In [1]:
import kimports; reload(kimports); from kimports import *
import kutils; reload(kutils)

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


<module 'kutils' from 'kutils.pyc'>

In [2]:
path = '../data/movielense/small'

In [53]:
ratings, movies, tags = [pd.read_csv(path + '/%s.csv' % f) for f in ['ratings', 'movies', 'tags']]

In [54]:
movies = movies.set_index('movieId')

In [63]:
movies.iloc[31]['title']

'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)'

In [71]:
movies.loc[164979]

title     Women of '69, Unboxed
genres              Documentary
Name: 164979, dtype: object

In [72]:
# Map from movie and user indexes to contiguous indexes,
# to reduce the size of the embedding layers
userid2idx = {o:i for i,o in enumerate(ratings['userId'].unique())}
movieid2idx = {o:i for i,o in enumerate(ratings['movieId'].unique())}
ratings['seqMovieId'] = ratings['movieId'].apply(lambda x: movieid2idx[x])
ratings['title'] = ratings['movieId'].apply(lambda x: movies.loc[x]['title'])
ratings['seqUserId'] = ratings['userId'].apply(lambda x: userid2idx[x])

In [73]:
num_factors = 50
mask = np.random.rand(len(ratings)) < 0.8
train_ratings = ratings[mask]
valid_ratings = ratings[~mask]

In [74]:
def fit(model, nb_epoch, lr):
    model.optimizer.lr = lr
    model.fit([train_ratings['seqUserId'], train_ratings['seqMovieId']], train_ratings['rating'],
          validation_data=([valid_ratings['seqUserId'], valid_ratings['seqMovieId']], valid_ratings['rating']),
          batch_size=64, nb_epoch=nb_epoch)

# Basic, which over fits

In [7]:
movie_input = keras.layers.Input(shape=(1,), dtype='int64', name='movie_input')
user_input = keras.layers.Input(shape=(1,), dtype='int64', name='user_input')

user_embedding = keras.layers.Embedding(ratings['seqUserId'].nunique(), num_factors, input_length=1)(user_input)
movie_embedding = keras.layers.Embedding(ratings['seqMovieId'].nunique(), num_factors, input_length=1)(movie_input)

layers = keras.layers.merge([user_embedding, movie_embedding], mode='dot')
layers = keras.layers.Flatten()(layers)
model = keras.models.Model([user_input, movie_input], layers)
model.compile(optimizer=keras.optimizers.Adam(0.001), loss='mse')

In [8]:
fit(model, 1, 0.001)

Train on 80080 samples, validate on 19924 samples
Epoch 1/1
80080/80080 [==============================] - 4s - loss: 10.7301 - val_loss: 4.2921


In [9]:
fit(model, 10, 0.01)

Train on 80080 samples, validate on 19924 samples
Epoch 1/10
80080/80080 [==============================] - 4s - loss: 2.4304 - val_loss: 1.8774
Epoch 2/10
80080/80080 [==============================] - 4s - loss: 1.2726 - val_loss: 1.4796
Epoch 3/10
80080/80080 [==============================] - 4s - loss: 0.9480 - val_loss: 1.3465
Epoch 4/10
80080/80080 [==============================] - 4s - loss: 0.7905 - val_loss: 1.3002
Epoch 5/10
80080/80080 [==============================] - 4s - loss: 0.6895 - val_loss: 1.2787
Epoch 6/10
80080/80080 [==============================] - 4s - loss: 0.6127 - val_loss: 1.2720
Epoch 7/10
80080/80080 [==============================] - 4s - loss: 0.5483 - val_loss: 1.2795
Epoch 8/10
80080/80080 [==============================] - 4s - loss: 0.4935 - val_loss: 1.2817
Epoch 9/10
80080/80080 [==============================] - 4s - loss: 0.4433 - val_loss: 1.2904
Epoch 10/10
80080/80080 [==============================] - 4s - loss: 0.3959 - val_loss: 1.2993

# Batch norm

In [10]:
movie_input = keras.layers.Input(shape=(1,), dtype='int64', name='movie_input')
user_input = keras.layers.Input(shape=(1,), dtype='int64', name='user_input')

user_embedding = keras.layers.Embedding(ratings['seqUserId'].nunique(), num_factors, input_length=1)(user_input)
movie_embedding = keras.layers.Embedding(ratings['seqMovieId'].nunique(), num_factors, input_length=1)(movie_input)

layers = keras.layers.merge([user_embedding, movie_embedding], mode='dot')
layers = keras.layers.normalization.BatchNormalization()(layers)
layers = keras.layers.Flatten()(layers)
layers = keras.layers.core.Dropout(0.05)(layers)
model = keras.models.Model([user_input, movie_input], layers)
model.compile(optimizer=keras.optimizers.Adam(0.001), loss='mse')

In [11]:
fit(model, 1, 0.001)

Train on 80080 samples, validate on 19924 samples
Epoch 1/1
80080/80080 [==============================] - 6s - loss: 10.1969 - val_loss: 6.8933


In [12]:
fit(model, 10, 0.01)

Train on 80080 samples, validate on 19924 samples
Epoch 1/10
80080/80080 [==============================] - 6s - loss: 4.5548 - val_loss: 3.1339
Epoch 2/10
80080/80080 [==============================] - 6s - loss: 1.9816 - val_loss: 1.5548
Epoch 3/10
80080/80080 [==============================] - 6s - loss: 1.0963 - val_loss: 1.1004
Epoch 4/10
80080/80080 [==============================] - 6s - loss: 0.9163 - val_loss: 1.0669
Epoch 5/10
80080/80080 [==============================] - 6s - loss: 0.8697 - val_loss: 1.0634
Epoch 6/10
80080/80080 [==============================] - 6s - loss: 0.8487 - val_loss: 1.0740
Epoch 7/10
80080/80080 [==============================] - 6s - loss: 0.8249 - val_loss: 1.0814
Epoch 8/10
80080/80080 [==============================] - 6s - loss: 0.8204 - val_loss: 1.0898
Epoch 9/10
80080/80080 [==============================] - 6s - loss: 0.7946 - val_loss: 1.0951
Epoch 10/10
80080/80080 [==============================] - 6s - loss: 0.7991 - val_loss: 1.1044

# Regularization in embedding layers

In [13]:
movie_input = keras.layers.Input(shape=(1,), dtype='int64', name='movie_input')
user_input = keras.layers.Input(shape=(1,), dtype='int64', name='user_input')

user_embedding = keras.layers.Embedding(ratings['seqUserId'].nunique(), num_factors, input_length=1, W_regularizer=keras.regularizers.l2(0.01))(user_input)
movie_embedding = keras.layers.Embedding(ratings['seqMovieId'].nunique(), num_factors, input_length=1, W_regularizer=keras.regularizers.l2(0.01))(movie_input)

layers = keras.layers.merge([user_embedding, movie_embedding], mode='dot')
layers = keras.layers.normalization.BatchNormalization()(layers)
layers = keras.layers.Flatten()(layers)
model = keras.models.Model([user_input, movie_input], layers)
model.compile(optimizer=keras.optimizers.Adam(0.001), loss='mse')

In [14]:
fit(model, 1, 0.001)

Train on 80080 samples, validate on 19924 samples
Epoch 1/1
80080/80080 [==============================] - 8s - loss: 10.4005 - val_loss: 6.6510


In [15]:
fit(model, 10, 0.01)

Train on 80080 samples, validate on 19924 samples
Epoch 1/10
80080/80080 [==============================] - 8s - loss: 5.0082 - val_loss: 2.9806
Epoch 2/10
80080/80080 [==============================] - 8s - loss: 2.4809 - val_loss: 1.4097
Epoch 3/10
80080/80080 [==============================] - 8s - loss: 1.5419 - val_loss: 0.9733
Epoch 4/10
80080/80080 [==============================] - 8s - loss: 1.3447 - val_loss: 0.9366
Epoch 5/10
80080/80080 [==============================] - 8s - loss: 1.3267 - val_loss: 0.9349
Epoch 6/10
80080/80080 [==============================] - 8s - loss: 1.3091 - val_loss: 0.9427
Epoch 7/10
80080/80080 [==============================] - 8s - loss: 1.3024 - val_loss: 0.9369
Epoch 8/10
80080/80080 [==============================] - 8s - loss: 1.2961 - val_loss: 0.9392
Epoch 9/10
80080/80080 [==============================] - 8s - loss: 1.2903 - val_loss: 0.9275
Epoch 10/10
80080/80080 [==============================] - 8s - loss: 1.2829 - val_loss: 0.9259

# Bias units

In [75]:
movie_input = keras.layers.Input(shape=(1,), dtype='int64', name='movie_input')
user_input = keras.layers.Input(shape=(1,), dtype='int64', name='user_input')

user_embedding = keras.layers.Embedding(ratings['seqUserId'].nunique(), num_factors, input_length=1, W_regularizer=keras.regularizers.l2(0.001))(user_input)
movie_embedding = keras.layers.Embedding(ratings['seqMovieId'].nunique(), num_factors, input_length=1, W_regularizer=keras.regularizers.l2(0.001))(movie_input)
b_user_embedding = keras.layers.Embedding(ratings['seqUserId'].nunique(), 1, input_length=1)(user_input)
b_movie_embedding = keras.layers.Embedding(ratings['seqMovieId'].nunique(), 1, input_length=1)(movie_input)
b_user_embedding = keras.layers.Flatten()(b_user_embedding)
b_movie_embedding = keras.layers.Flatten()(b_movie_embedding)
layers = keras.layers.merge([user_embedding, movie_embedding], mode='dot')
layers = keras.layers.Flatten()(layers)
layers = keras.layers.merge([layers, b_user_embedding], mode='sum')
layers = keras.layers.merge([layers, b_movie_embedding], mode='sum')


#layers = keras.layers.normalization.BatchNormalization()(layers)

model = keras.models.Model([user_input, movie_input], layers)
model.compile(optimizer=keras.optimizers.Adam(0.001), loss='mse')

In [76]:
fit(model, 1, 0.001)

Train on 80031 samples, validate on 19973 samples
Epoch 1/1
80031/80031 [==============================] - 6s - loss: 10.1590 - val_loss: 4.7631


In [77]:
fit(model, 10, 0.01)

Train on 80031 samples, validate on 19973 samples
Epoch 1/10
80031/80031 [==============================] - 6s - loss: 6.2312 - val_loss: 3.3124
Epoch 2/10
80031/80031 [==============================] - 6s - loss: 5.1941 - val_loss: 2.8437
Epoch 3/10
80031/80031 [==============================] - 6s - loss: 4.2102 - val_loss: 2.4925
Epoch 4/10
80031/80031 [==============================] - 6s - loss: 3.2844 - val_loss: 2.1728
Epoch 5/10
80031/80031 [==============================] - 6s - loss: 2.4724 - val_loss: 1.8651
Epoch 6/10
80031/80031 [==============================] - 6s - loss: 1.8221 - val_loss: 1.5662
Epoch 7/10
80031/80031 [==============================] - 6s - loss: 1.3851 - val_loss: 1.3153
Epoch 8/10
80031/80031 [==============================] - 6s - loss: 1.1540 - val_loss: 1.1595
Epoch 9/10
80031/80031 [==============================] - 6s - loss: 1.0292 - val_loss: 1.0676
Epoch 10/10
80031/80031 [==============================] - 6s - loss: 0.9502 - val_loss: 1.0074

# Look at stuffs

## First the movie bias

In [90]:
bias_model = keras.models.Model(movie_input, b_movie_embedding)
bias_preds = bias_model.predict(ratings['seqMovieId'].unique())

In [94]:
movie_ratings = sorted(zip(ratings['title'].unique(), bias_preds), key=lambda x: x[1])

In [95]:
movie_ratings[:10]

[('Spy Kids 3-D: Game Over (2003)', array([-0.57035518], dtype=float32)),
 ("Joe's Apartment (1996)", array([-0.511823], dtype=float32)),
 ('Rage: Carrie 2, The (1999)', array([-0.47340706], dtype=float32)),
 ('Baby Geniuses (1999)', array([-0.44225287], dtype=float32)),
 ('Beverly Hills Chihuahua (2008)', array([-0.43715361], dtype=float32)),
 ('In the Army Now (1994)', array([-0.42980075], dtype=float32)),
 ('Vampires Suck (2010)', array([-0.42295581], dtype=float32)),
 ('Carnosaur 3: Primal Species (1996)', array([-0.40701455], dtype=float32)),
 ('Jaws: The Revenge (1987)', array([-0.40035567], dtype=float32)),
 ('Tekkonkinkreet (Tekkon kinkur\xc3\xaeto) (2006)',
  array([-0.3926492], dtype=float32))]

In [96]:
movie_ratings[-10:]

[('Forrest Gump (1994)', array([ 2.00717521], dtype=float32)),
 ('Silence of the Lambs, The (1991)', array([ 2.01407266], dtype=float32)),
 ('Godfather: Part II, The (1974)', array([ 2.02144289], dtype=float32)),
 ('Fargo (1996)', array([ 2.07270026], dtype=float32)),
 ('Star Wars: Episode IV - A New Hope (1977)',
  array([ 2.07493281], dtype=float32)),
 ("Schindler's List (1993)", array([ 2.09836292], dtype=float32)),
 ('Usual Suspects, The (1995)', array([ 2.1232934], dtype=float32)),
 ('Pulp Fiction (1994)', array([ 2.16603613], dtype=float32)),
 ('Godfather, The (1972)', array([ 2.2109468], dtype=float32)),
 ('Shawshank Redemption, The (1994)', array([ 2.42214465], dtype=float32))]